In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ
%env CLEARML_API_SECRET_KEY=if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g

In [ ]:
import os

# 设置环境变量
os.environ["CLEARML_WEB_HOST"] = "https://app.clear.ml"
os.environ["CLEARML_API_HOST"] = "https://api.clear.ml"
os.environ["CLEARML_FILES_HOST"] = "https://files.clear.ml"
os.environ["CLEARML_API_ACCESS_KEY"] = "N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ"
os.environ["CLEARML_API_SECRET_KEY"] = "if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g"

from clearml import Task

try:
    task = Task.init(project_name="Test", task_name="Test")
    print("连接成功，任务ID:", task.id)
    task.close()
except Exception as e:
    print("连接失败，错误信息:", e)

In [ ]:
"""
数据预处理任务 - 基于你的adult_model_clear_ml.ipynb代码
创建独立的ClearML Task和Dataset版本
"""

# ==================== 1. 导入库 ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import json

# ==================== 2. 设置ClearML凭证 ====================
# 使用你已有的配置方式
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ
%env CLEARML_API_SECRET_KEY=if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g

# ==================== 3. 初始化数据预处理Task ====================
from clearml import Task, Dataset

# 创建独立的数据预处理任务
print("初始化独立的数据预处理任务...")
task_data_prep = Task.init(
    project_name='Adult Income - Курсовая',  # 项目名称
    task_name='Data Preprocessing - Task 1',  # 任务名称
    task_type=Task.TaskTypes.data_processing,  # 明确指定为数据处理任务
    tags=['data-processing', 'preprocessing', 'курсовая']
)

print(f"✅ 数据预处理任务创建成功!")
print(f"📋 Task ID: {task_data_prep.id}")

# ==================== 4. 加载原始数据 ====================
print("\n" + "="*50)
print("步骤1: 加载原始数据")

fpath = 'adults.csv'
df_raw = pd.read_csv(fpath)

print(f"✅ 数据加载成功!")
print(f"📊 原始数据形状: {df_raw.shape}")
print(f"📋 前3行数据预览:")
print(df_raw.head(3))

# ==================== 5. 上传原始数据作为Artifact ====================
print("\n" + "="*50)
print("步骤2: 上传原始数据到ClearML")

task_data_prep.upload_artifact(
    name='raw_data.csv',
    artifact_object=fpath,
    metadata={
        'description': '原始Adult Income数据集',
        'rows': df_raw.shape[0],
        'columns': df_raw.shape[1]
    }
)
print("✅ 原始数据已上传为Artifact")

# ==================== 6. 数据预处理（基于你的代码） ====================
print("\n" + "="*50)
print("步骤3: 数据预处理")

# 记录预处理步骤
preprocessing_steps = []

# 6.1 删除不相关特征（基于你的代码）
print("\n1. 删除不相关特征...")
df = df_raw.drop(['fnlwgt', 'education_num'], axis=1)
print(f"✅ 已删除 'fnlwgt' 和 'education_num' 列")
preprocessing_steps.append("删除 fnlwgt 和 education_num 列")

# 6.2 处理缺失值（基于你的代码）
print("\n2. 处理缺失值...")
col_names = df.columns
for c in col_names:
    df = df.replace("?", np.nan)
    
# 用众数填充缺失值
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
print("✅ 已处理缺失值（'?'替换并用众数填充）")
preprocessing_steps.append("处理缺失值")

# 6.3 简化分类变量（基于你的代码）
print("\n3. 简化分类变量...")
df.replace(['Divorced', 'Married-AF-spouse',
            'Married-civ-spouse', 'Married-spouse-absent',
            'Never-married', 'Separated', 'Widowed'],
           ['divorced', 'married', 'married', 'married',
            'not married', 'not married', 'not married'], inplace=True)
print("✅ 已简化婚姻状态变量")
preprocessing_steps.append("简化婚姻状态变量")

# 6.4 编码分类变量（基于你的代码）
print("\n4. 编码分类变量...")
category_col = ['workclass', 'race', 'education', 'marital_status', 'occupation',
                'relationship', 'sex', 'native_country', 'class']
labelEncoder = LabelEncoder()

mapping_dict = {}
for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])
    
    le_name_mapping = dict(zip(labelEncoder.classes_,
                               labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col] = le_name_mapping

print(f"✅ 已编码 {len(category_col)} 个分类变量")
preprocessing_steps.append("编码分类变量")

# 6.5 保存编码映射
print("\n5. 保存编码映射...")
encoding_map_file = 'encoding_mappings.json'

# 自定义序列化器，将NumPy类型转换为Python内置类型
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.floating)):
            return obj.item()  # 转换为Python内置类型
        return super().default(obj)

with open(encoding_map_file, 'w') as f:
    json.dump(mapping_dict, f, indent=2, cls=NumpyEncoder)
# ==================== 7. 数据分割（基于你的代码） ====================
print("\n" + "="*50)
print("步骤4: 数据分割")

# 准备特征和目标变量
X = df.values[:, 0:12]
Y = df.values[:, 12]

# 分割数据（使用你的参数）
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=100
)

print(f"✅ 数据分割完成:")
print(f"   训练集: {X_train.shape[0]} 样本, {X_train.shape[1]} 特征")
print(f"   测试集: {X_test.shape[0]} 样本, {X_test.shape[1]} 特征")

# 计算类别分布
train_class_0 = (y_train == 0).sum()
train_class_1 = (y_train == 1).sum()
train_total = len(y_train)

print(f"   训练集类别分布:")
print(f"     类别0 (<=50K): {train_class_0} ({(train_class_0/train_total)*100:.1f}%)")
print(f"     类别1 (>50K): {train_class_1} ({(train_class_1/train_total)*100:.1f}%)")

# ==================== 8. 保存处理后的数据 ====================
print("\n" + "="*50)
print("步骤5: 保存处理后的数据")

import pickle

# 保存处理后的完整数据集
processed_df_file = 'adult_processed_full.pkl'
with open(processed_df_file, 'wb') as f:
    pickle.dump(df, f)

# 保存训练集和测试集
train_data_file = 'adult_train_data.pkl'
test_data_file = 'adult_test_data.pkl'

train_data = {'X': X_train, 'y': y_train}
test_data = {'X': X_test, 'y': y_test}

with open(train_data_file, 'wb') as f:
    pickle.dump(train_data, f)

with open(test_data_file, 'wb') as f:
    pickle.dump(test_data, f)

print("✅ 处理后的数据已保存为 pickle 文件")

# ==================== 9. 上传处理后的数据作为Artifact ====================
print("\n" + "="*50)
print("步骤6: 上传处理后的数据到ClearML")

# 上传处理后的完整数据集
task_data_prep.upload_artifact(
    name='processed_data_full.pkl',
    artifact_object=processed_df_file
)

# 上传训练集
task_data_prep.upload_artifact(
    name='train_data.pkl',
    artifact_object=train_data_file
)

# 上传测试集
task_data_prep.upload_artifact(
    name='test_data.pkl',
    artifact_object=test_data_file
)

# 上传编码映射
task_data_prep.upload_artifact(
    name='encoding_mappings.json',
    artifact_object=encoding_map_file
)

print("✅ 所有处理后的数据已上传为Artifact")

# ==================== 10. 创建Dataset版本 ====================
print("\n" + "="*50)
print("步骤7: 创建ClearML Dataset版本")

# Dataset版本1.0：完整处理后的数据
print("创建 Dataset v1.0 (完整处理后的数据)...")
dataset_v1 = Dataset.create(
    dataset_name='adult_income_processed',
    dataset_project='Adult Income - Курсовая',
    dataset_version='1.0',
    description='Adult Income数据集完整预处理后的版本，包含清洗和编码'
)

# 添加文件到Dataset
dataset_v1.add_files(processed_df_file)
dataset_v1.add_files(encoding_map_file)

# 上传并完成Dataset
dataset_v1.upload()
dataset_v1.finalize()

print(f"✅ Dataset v1.0 创建成功!")
print(f"   Dataset ID: {dataset_v1.id}")

# Dataset版本2.0：训练/测试分割
print("\n创建 Dataset v2.0 (训练/测试分割)...")
dataset_v2 = Dataset.create(
    dataset_name='adult_income_splits',
    dataset_project='Adult Income - Курсовая',
    dataset_version='2.0',
    parent_datasets=[dataset_v1.id],  # 指定父版本
    description='基于v1.0分割的训练集和测试集'
)

# 添加文件到Dataset
dataset_v2.add_files(train_data_file)
dataset_v2.add_files(test_data_file)
dataset_v2.add_files(encoding_map_file)

# 上传并完成Dataset
dataset_v2.upload()
dataset_v2.finalize()

print(f"✅ Dataset v2.0 创建成功!")
print(f"   Dataset ID: {dataset_v2.id}")

# ==================== 11. 记录配置和结果 ====================
print("\n" + "="*50)
print("步骤8: 记录配置和结果")

# 创建配置字典
config = {
    'preprocessing': {
        'steps': preprocessing_steps,
        'dropped_columns': ['fnlwgt', 'education_num'],
        'test_size': 0.3,
        'random_state': 100,
        'encoded_columns': category_col
    },
    'data_statistics': {
        'original_shape': df_raw.shape,
        'processed_shape': df.shape,
        'train_samples': X_train.shape[0],
        'test_samples': X_test.shape[0],
        'features': X_train.shape[1],
        'class_distribution_train': {
            'class_0': int(train_class_0),
            'class_1': int(train_class_1),
            'class_0_percent': float((train_class_0/train_total)*100),
            'class_1_percent': float((train_class_1/train_total)*100)
        }
    },
    'datasets': {
        'v1.0': dataset_v1.id,
        'v2.0': dataset_v2.id
    }
}

# 连接到Task配置
task_data_prep.connect(config)
print("✅ 配置已记录到ClearML")

# ==================== 12. 生成报告 ====================
print("\n" + "="*50)
print("步骤9: 生成报告")

# 获取logger
logger = task_data_prep.get_logger()

# 报告数据统计
logger.report_table(
    title="数据预处理统计",
    series="summary",
    table_plot=[
        ["指标", "原始数据", "处理后数据"],
        ["样本数", df_raw.shape[0], df.shape[0]],
        ["特征数", df_raw.shape[1], df.shape[1]],
        ["训练集样本", "-", X_train.shape[0]],
        ["测试集样本", "-", X_test.shape[0]]
    ]
)

# 报告类别分布
logger.report_table(
    title="训练集类别分布",
    series="class_distribution",
    table_plot=[
        ["类别", "数量", "百分比"],
        ["<=50K", int(train_class_0), f"{(train_class_0/train_total)*100:.1f}%"],
        [">50K", int(train_class_1), f"{(train_class_1/train_total)*100:.1f}%"]
    ]
)

# 报告预处理步骤
steps_table = [["步骤", "描述"]]
for i, step in enumerate(preprocessing_steps, 1):
    steps_table.append([f"步骤 {i}", step])

logger.report_table(
    title="预处理步骤",
    series="steps",
    table_plot=steps_table
)

print("✅ 报告已生成")

# ==================== 13. 清理临时文件 ====================
print("\n" + "="*50)
print("步骤10: 清理临时文件")

import os
temp_files = [processed_df_file, train_data_file, test_data_file, encoding_map_file]
for file in temp_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"🗑️  已删除临时文件: {file}")

# ==================== 14. 完成任务 ====================
print("\n" + "="*50)
print("🎉 数据预处理任务完成!")

print(f"\n📊 任务总结:")
print(f"   Task ID: {task_data_prep.id}")
print(f"   Task链接: {task_data_prep.get_output_log_web_page()}")
print(f"   Dataset版本创建:")
print(f"     - v1.0 (完整数据): {dataset_v1.id}")
print(f"     - v2.0 (训练/测试): {dataset_v2.id}")
print(f"   预处理步骤: {len(preprocessing_steps)} 个")
print(f"   最终特征数: {X_train.shape[1]}")

# 关闭任务
task_data_prep.close()
print("\n✅ 任务已关闭")
print("📋 请访问ClearML Web界面查看完整结果")

In [ ]:
"""
实验 1: DecisionTreeClassifier 模型训练
基于你的原有代码，集成到 ClearML 实验中
"""

# ==================== 1. 导入库 ====================
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
import json

# ==================== 2. 设置ClearML凭证 ====================
# 使用你已有的配置方式
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ
%env CLEARML_API_SECRET_KEY=if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g

# ==================== 3. 初始化ML实验Task ====================
from clearml import Task, Dataset

print("初始化第一个ML实验任务...")
task_ml1 = Task.init(
    project_name='Adult Income - Курсовая',
    task_name='ML Experiment 1 - DecisionTree',
    task_type=Task.TaskTypes.training,
    tags=['decision-tree', 'gini', 'baseline', 'курсовая']
)

print(f"✅ ML实验任务创建成功!")
print(f"📋 Task ID: {task_ml1.id}")

# ==================== 4. 加载Dataset v2.0 ====================
print("\n" + "="*50)
print("步骤1: 加载Dataset v2.0 (训练/测试分割)")

try:
    # 获取Dataset v2.0
    dataset = Dataset.get(
        dataset_name='adult_income_splits',
        dataset_project='Adult Income - Курсовая',
        dataset_version='2.0'
    )
    
    # 获取本地副本路径
    dataset_path = dataset.get_local_copy()
    print(f"✅ Dataset v2.0 加载成功!")
    print(f"📂 本地路径: {dataset_path}")
    
except Exception as e:
    print(f"❌ Dataset加载失败: {e}")
    print("请确保数据预处理任务已完成并创建了Dataset v2.0")
    exit()

# ==================== 5. 加载训练和测试数据 ====================
print("\n" + "="*50)
print("步骤2: 加载训练和测试数据")

# 加载训练数据
train_data_file = os.path.join(dataset_path, 'adult_train_data.pkl')
with open(train_data_file, 'rb') as f:
    train_data = pickle.load(f)

X_train = np.array(train_data['X'])
y_train = np.array(train_data['y'])

# 加载测试数据
test_data_file = os.path.join(dataset_path, 'adult_test_data.pkl')
with open(test_data_file, 'rb') as f:
    test_data = pickle.load(f)

X_test = np.array(test_data['X'])
y_test = np.array(test_data['y'])

print(f"✅ 数据加载完成:")
print(f"   训练集: {X_train.shape[0]} 样本, {X_train.shape[1]} 特征")
print(f"   测试集: {X_test.shape[0]} 样本")

# ==================== 6. 设置并记录超参数 ====================
print("\n" + "="*50)
print("步骤3: 设置并记录超参数")

# 基于你的原始代码设置超参数
hyperparams = {
    'model_type': 'DecisionTreeClassifier',
    'criterion': 'gini',
    'max_depth': 5,
    'min_samples_leaf': 5,
    'random_state': 100,
    'test_size': 0.3,
    'dataset_version': '2.0',
    'features_used': X_train.shape[1]
}

# 连接到Task配置 (这将出现在ClearML的Configuration标签页)
task_ml1.connect(hyperparams)
print("✅ 超参数已记录到ClearML")

print("\n📋 使用的超参数:")
for key, value in hyperparams.items():
    print(f"   {key}: {value}")

# ==================== 7. 训练模型 ====================
print("\n" + "="*50)
print("步骤4: 训练DecisionTree模型")

print("开始训练模型...")
model = DecisionTreeClassifier(
    criterion=hyperparams['criterion'],
    max_depth=hyperparams['max_depth'],
    min_samples_leaf=hyperparams['min_samples_leaf'],
    random_state=hyperparams['random_state']
)

# 训练模型
model.fit(X_train, y_train)
print("✅ 模型训练完成!")

# 获取模型特征重要性
feature_importance = model.feature_importances_
print(f"\n🔍 特征重要性 (前5个):")
for i, imp in enumerate(feature_importance[:5]):
    print(f"   特征 {i}: {imp:.4f}")

# ==================== 8. 模型评估 ====================
print("\n" + "="*50)
print("步骤5: 模型评估")

# 预测
y_pred = model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred) * 100

# 计算其他指标
from sklearn.metrics import precision_score, recall_score, f1_score

# 获取预测概率（用于AUC计算）
y_pred_proba = model.predict_proba(X_test)[:, 1]

# 计算AUC
try:
    from sklearn.metrics import roc_auc_score
    auc_score = roc_auc_score(y_test, y_pred_proba) * 100
    print(f"   AUC分数: {auc_score:.2f}%")
except Exception as e:
    auc_score = None
    print(f"   AUC计算失败: {e}")
precision = precision_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100

print(f"📊 模型评估结果:")
print(f"   准确率 (Accuracy): {accuracy:.2f}%")
print(f"   精确率 (Precision): {precision:.2f}%")
print(f"   召回率 (Recall): {recall:.2f}%")
print(f"   F1分数: {f1:.2f}%")

# 生成分类报告
print(f"\n📋 分类报告:")
report = classification_report(y_test, y_pred, target_names=['<=50K', '>50K'])
print(report)

# ==================== 9. 记录指标到ClearML ====================
print("\n" + "="*50)
print("步骤6: 记录指标到ClearML")

# 获取logger
logger = task_ml1.get_logger()

# 记录单值指标 (将出现在RESULTS → SCALARS)
logger.report_single_value(name="test_accuracy", value=accuracy)
logger.report_single_value(name="test_precision", value=precision)
logger.report_single_value(name="test_recall", value=recall)
logger.report_single_value(name="test_f1", value=f1)

# 记录AUC分数（如果计算成功）
if auc_score is not None:
    logger.report_single_value(name="test_auc", value=auc_score)
    print("✅ AUC已记录到ClearML")

print("✅ 指标已记录到ClearML (RESULTS → SCALARS)")

# 记录分类报告作为表格
print("\n记录分类报告表格...")
report_dict = classification_report(y_test, y_pred, target_names=['<=50K', '>50K'], output_dict=True)

# 创建表格数据
table_data = [["类别", "精确率", "召回率", "F1分数", "支持度"]]
for class_name in ['<=50K', '>50K']:
    metrics = report_dict[class_name]
    table_data.append([
        class_name,
        f"{metrics['precision']*100:.2f}%",
        f"{metrics['recall']*100:.2f}%",
        f"{metrics['f1-score']*100:.2f}%",
        int(metrics['support'])
    ])

# 添加加权平均
metrics = report_dict['weighted avg']
table_data.append([
    "加权平均",
    f"{metrics['precision']*100:.2f}%",
    f"{metrics['recall']*100:.2f}%",
    f"{metrics['f1-score']*100:.2f}%",
    int(report_dict['macro avg']['support'] * 2)  # 总样本数
])

logger.report_table(
    title="分类报告详情",
    series="decision_tree",
    table_plot=table_data
)

# ==================== 10. 记录混淆矩阵 ====================
print("\n记录混淆矩阵...")
cm = confusion_matrix(y_test, y_pred)

# 创建混淆矩阵可视化
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['<=50K', '>50K'],
            yticklabels=['<=50K', '>50K'])
plt.title('混淆矩阵 - DecisionTree')
plt.ylabel('真实标签')
plt.xlabel('预测标签')
plt.tight_layout()

# 使用report_matplotlib_figure方法直接报告图形
logger.report_matplotlib_figure(
    title="混淆矩阵",
    series="confusion_matrix",
    figure=plt.gcf(),
    report_interactive=False
)

# 保存图像文件（可选）
cm_image_path = 'confusion_matrix.png'
plt.savefig(cm_image_path, dpi=100, bbox_inches='tight')
plt.close()

print("✅ 混淆矩阵已记录")

# 上传混淆矩阵图像作为Artifact
task_ml1.upload_artifact(
    name='confusion_matrix.png',
    artifact_object=cm_image_path
)

# 在记录混淆矩阵的部分之后，添加ROC曲线
print("\n记录ROC曲线...")
from sklearn.metrics import roc_curve

# 计算ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# 绘制ROC曲线
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC曲线 (AUC = {auc_score:.2f}%)')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='随机猜测')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('假正率 (FPR)')
plt.ylabel('真正率 (TPR)')
plt.title('ROC曲线 - DecisionTree')
plt.legend(loc="lower right")
plt.grid(True)

# 记录到ClearML
logger.report_matplotlib_figure(
    title="ROC曲线",
    series="roc_curve",
    figure=plt.gcf(),
    report_interactive=False
)

# 保存图像
roc_image_path = 'roc_curve_dt.png'
plt.savefig(roc_image_path, dpi=100, bbox_inches='tight')
plt.close()

print("✅ ROC曲线已记录")

# 上传ROC曲线图像作为Artifact
task_ml1.upload_artifact(
    name='roc_curve_dt.png',
    artifact_object=roc_image_path
)

# ==================== 11. 保存模型为pkl文件 ====================
print("\n" + "="*50)
print("步骤7: 保存模型为pkl文件")

# 保存模型
model_filename = 'decision_tree_model_v1.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ 模型已保存为: {model_filename}")

# 保存特征重要性
feature_importance_dict = {
    'feature_importances': feature_importance.tolist(),
    'top_features': []
}

# 识别最重要的特征
indices = np.argsort(feature_importance)[::-1]
for i, idx in enumerate(indices[:10]):  # 前10个最重要特征
    feature_importance_dict['top_features'].append({
        'feature_index': int(idx),
        'importance': float(feature_importance[idx])
    })

importance_filename = 'feature_importance.json'
with open(importance_filename, 'w') as f:
    json.dump(feature_importance_dict, f, indent=2)

print(f"✅ 特征重要性已保存为: {importance_filename}")

# ==================== 12. 上传模型和文件作为Artifact ====================
print("\n" + "="*50)
print("步骤8: 上传模型和文件到ClearML")

# 上传模型
task_ml1.upload_artifact(
    name='decision_tree_model.pkl',
    artifact_object=model_filename,
    metadata={
        'model_type': 'DecisionTreeClassifier',
        'accuracy': accuracy,
        'hyperparameters': hyperparams
    }
)

# 上传特征重要性
task_ml1.upload_artifact(
    name='feature_importance.json',
    artifact_object=importance_filename
)

# 上传混淆矩阵图像
task_ml1.upload_artifact(
    name='confusion_matrix.png',
    artifact_object=cm_image_path
)

print("✅ 模型和文件已上传为Artifact")

# ==================== 13. 记录实验总结 ====================
print("\n" + "="*50)
print("步骤9: 记录实验总结")

# 创建实验总结
experiment_summary = {
    'experiment_info': {
        'task_id': task_ml1.id,
        'model': 'DecisionTreeClassifier',
        'dataset_version': '2.0',
        'timestamp': pd.Timestamp.now().isoformat()
    },
    'performance_metrics': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    },
    'model_info': {
        'n_features': X_train.shape[1],
        'n_classes': len(np.unique(y_train)),
        'tree_depth': model.get_depth(),
        'n_leaves': model.get_n_leaves()
    },
    'hyperparameters_used': hyperparams
}

# 记录总结表格
summary_table = [
    ["指标", "值"],
    ["准确率", f"{accuracy:.2f}%"],
    ["精确率", f"{precision:.2f}%"],
    ["召回率", f"{recall:.2f}%"],
    ["F1分数", f"{f1:.2f}%"],
    ["AUC分数", f"{auc_score:.2f}%" if auc_score is not None else "N/A"],
    ["特征数量", X_train.shape[1]],
    ["树深度", model.get_depth()],
    ["叶子节点数", model.get_n_leaves()],
    ["数据集版本", "2.0"],
    ["测试集大小", f"{X_test.shape[0]} 样本"]
]

logger.report_table(
    title="实验总结",
    series="summary",
    table_plot=summary_table
)

# ==================== 14. 清理临时文件 ====================
print("\n" + "="*50)
print("步骤10: 清理临时文件")

import os
temp_files = [model_filename, importance_filename, cm_image_path]
for file in temp_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"🗑️  已删除临时文件: {file}")

# ==================== 15. 完成任务 ====================
print("\n" + "="*50)
print("🎉 第一个ML实验完成!")

print(f"\n📊 实验总结:")
print(f"   任务ID: {task_ml1.id}")
print(f"   任务链接: {task_ml1.get_output_log_web_page()}")
print(f"   模型: DecisionTreeClassifier")
print(f"   准确率: {accuracy:.2f}%")
print(f"   数据集版本: 2.0")
print(f"   超参数:")
for key, value in hyperparams.items():
    if key not in ['test_size', 'dataset_version', 'features_used']:
        print(f"     {key}: {value}")

print(f"\n📋 在ClearML中可以查看:")
print(f"   - Configuration: 超参数")
print(f"   - Artifacts: 模型文件 (.pkl)")
print(f"   - RESULTS → SCALARS: 准确率等指标")
print(f"   - RESULTS → PLOTS: 混淆矩阵")

# 关闭任务
task_ml1.close()
print("\n✅ 任务已关闭")
print("📋 请访问ClearML Web界面查看完整结果")

In [ ]:
"""
实验 2: RandomForestClassifier 模型训练
"""

# ==================== 1. 导入库 ====================
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
import pickle
import json
import matplotlib.pyplot as plt
import seaborn as sns

# ==================== 2. 设置ClearML凭证 ====================
# 使用你已有的配置方式
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ
%env CLEARML_API_SECRET_KEY=if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g

# ==================== 3. 初始化第二个ML实验Task ====================
from clearml import Task, Dataset

print("初始化第二个ML实验任务...")
task_ml2 = Task.init(
    project_name='Adult Income - Курсовая',
    task_name='ML Experiment 2 - RandomForest',
    task_type=Task.TaskTypes.training,
    tags=['random-forest', 'ensemble', 'comparison', 'курсовая']
)

print(f"✅ ML实验任务2创建成功!")
print(f"📋 Task ID: {task_ml2.id}")

# ==================== 4. 加载Dataset v2.0 ====================
print("\n" + "="*50)
print("步骤1: 加载Dataset v2.0 (与实验1相同)")

try:
    # 获取Dataset v2.0
    dataset = Dataset.get(
        dataset_name='adult_income_splits',
        dataset_project='Adult Income - Курсовая',
        dataset_version='2.0'
    )
    
    # 获取本地副本路径
    dataset_path = dataset.get_local_copy()
    print(f"✅ Dataset v2.0 加载成功!")
    print(f"📂 本地路径: {dataset_path}")
    
except Exception as e:
    print(f"❌ Dataset加载失败: {e}")
    print("请确保数据预处理任务已完成并创建了Dataset v2.0")
    exit()

# ==================== 5. 加载训练和测试数据 ====================
print("\n" + "="*50)
print("步骤2: 加载训练和测试数据")

# 加载训练数据
train_data_file = os.path.join(dataset_path, 'adult_train_data.pkl')
with open(train_data_file, 'rb') as f:
    train_data = pickle.load(f)

X_train = np.array(train_data['X'])
y_train = np.array(train_data['y'])

# 加载测试数据
test_data_file = os.path.join(dataset_path, 'adult_test_data.pkl')
with open(test_data_file, 'rb') as f:
    test_data = pickle.load(f)

X_test = np.array(test_data['X'])
y_test = np.array(test_data['y'])

print(f"✅ 数据加载完成:")
print(f"   训练集: {X_train.shape[0]} 样本, {X_train.shape[1]} 特征")
print(f"   测试集: {X_test.shape[0]} 样本")

# ==================== 6. 设置并记录超参数 ====================
print("\n" + "="*50)
print("步骤3: 设置并记录RandomForest超参数")

# RandomForest与DecisionTree不同的超参数
hyperparams = {
    'model_type': 'RandomForestClassifier',
    'n_estimators': 100,           # 树的数量
    'criterion': 'gini',           # 分割标准
    'max_depth': 10,               # 树的最大深度
    'min_samples_split': 2,        # 内部节点再划分所需最小样本数
    'min_samples_leaf': 1,         # 叶子节点最少样本数
    'max_features': 'sqrt',        # 考虑的最大特征数
    'bootstrap': True,             # 有放回抽样
    'oob_score': True,             # 使用袋外样本来评估模型
    'random_state': 42,            # 随机种子
    'n_jobs': -1,                  # 使用所有CPU核心
    'test_size': 0.3,
    'dataset_version': '2.0',
    'features_used': X_train.shape[1]
}

# 连接到Task配置
task_ml2.connect(hyperparams)
print("✅ 超参数已记录到ClearML")

print("\n📋 使用的主要超参数:")
for key, value in hyperparams.items():
    if key not in ['test_size', 'dataset_version', 'features_used']:
        print(f"   {key}: {value}")

# ==================== 7. 训练模型 ====================
print("\n" + "="*50)
print("步骤4: 训练RandomForest模型")

print("开始训练RandomForest模型...")
model = RandomForestClassifier(
    n_estimators=hyperparams['n_estimators'],
    criterion=hyperparams['criterion'],
    max_depth=hyperparams['max_depth'],
    min_samples_split=hyperparams['min_samples_split'],
    min_samples_leaf=hyperparams['min_samples_leaf'],
    max_features=hyperparams['max_features'],
    bootstrap=hyperparams['bootstrap'],
    oob_score=hyperparams['oob_score'],
    random_state=hyperparams['random_state'],
    n_jobs=hyperparams['n_jobs'],
    verbose=1  # 显示训练进度
)

# 训练模型
model.fit(X_train, y_train)
print("✅ 模型训练完成!")

# 获取OOB分数（如果可用）
if hasattr(model, 'oob_score_'):
    print(f"📊 OOB分数: {model.oob_score_:.4f}")

# 获取模型特征重要性
feature_importance = model.feature_importances_
print(f"\n🔍 特征重要性 (前10个):")
indices = np.argsort(feature_importance)[::-1]
for i, idx in enumerate(indices[:10]):
    print(f"   特征 {idx}: {feature_importance[idx]:.4f}")

# ==================== 8. 模型评估 ====================
print("\n" + "="*50)
print("步骤5: 模型评估")

# 预测
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # 预测概率（用于AUC）
# 计算ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr) * 100
# 计算准确率
accuracy = accuracy_score(y_test, y_pred) * 100

# 计算其他指标
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100

# 计算AUC（如果适用）
try:
    auc_score = roc_auc_score(y_test, y_pred_proba) * 100
except:
    auc_score = None

print(f"📊 模型评估结果:")
print(f"   准确率 (Accuracy): {accuracy:.2f}%")
print(f"   精确率 (Precision): {precision:.2f}%")
print(f"   召回率 (Recall): {recall:.2f}%")
print(f"   F1分数: {f1:.2f}%")
if auc_score:
    print(f"   AUC分数: {auc_score:.2f}%")

# 生成分类报告
print(f"\n📋 分类报告:")
report = classification_report(y_test, y_pred, target_names=['<=50K', '>50K'])
print(report)

# ==================== 9. 记录指标到ClearML ====================
print("\n" + "="*50)
print("步骤6: 记录指标到ClearML")

# 获取logger
logger = task_ml2.get_logger()

# 记录单值指标 (将出现在RESULTS → SCALARS)
logger.report_single_value(name="test_accuracy", value=accuracy)
logger.report_single_value(name="test_precision", value=precision)
logger.report_single_value(name="test_recall", value=recall)
logger.report_single_value(name="test_f1", value=f1)

if auc_score:
    logger.report_single_value(name="test_auc", value=auc_score)

# 记录OOB分数（如果可用）
if hasattr(model, 'oob_score_'):
    logger.report_single_value(name="oob_score", value=model.oob_score_ * 100)

print("✅ 指标已记录到ClearML (RESULTS → SCALARS)")

# 记录分类报告作为表格
print("\n记录分类报告表格...")
report_dict = classification_report(y_test, y_pred, target_names=['<=50K', '>50K'], output_dict=True)

# 创建表格数据
table_data = [["类别", "精确率", "召回率", "F1分数", "支持度"]]
for class_name in ['<=50K', '>50K']:
    metrics = report_dict[class_name]
    table_data.append([
        class_name,
        f"{metrics['precision']*100:.2f}%",
        f"{metrics['recall']*100:.2f}%",
        f"{metrics['f1-score']*100:.2f}%",
        int(metrics['support'])
    ])

# 添加加权平均
metrics = report_dict['weighted avg']
table_data.append([
    "加权平均",
    f"{metrics['precision']*100:.2f}%",
    f"{metrics['recall']*100:.2f}%",
    f"{metrics['f1-score']*100:.2f}%",
    int(report_dict['macro avg']['support'] * 2)
])

logger.report_table(
    title="分类报告详情",
    series="random_forest",
    table_plot=table_data
)

# ==================== 10. 记录混淆矩阵 ====================
print("\n记录混淆矩阵...")
cm = confusion_matrix(y_test, y_pred)

# 创建混淆矩阵可视化
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['<=50K', '>50K'],
            yticklabels=['<=50K', '>50K'])
plt.title('混淆矩阵 - RandomForest')
plt.ylabel('真实标签')
plt.xlabel('预测标签')
plt.tight_layout()

# 使用report_matplotlib_figure方法直接报告图形
logger.report_matplotlib_figure(
    title="混淆矩阵",
    series="confusion_matrix",
    figure=plt.gcf(),
    report_interactive=False
)

# 保存图像文件
cm_image_path = 'confusion_matrix_rf.png'
plt.savefig(cm_image_path, dpi=100, bbox_inches='tight')
plt.close()

print("✅ 混淆矩阵已记录")

# ==================== 11. 记录ROC曲线 ====================
print("\n" + "="*50)
print("步骤7: 记录ROC曲线")

# 绘制ROC曲线
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC曲线 (AUC = {roc_auc:.2f}%)')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='随机猜测')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('假正率 (False Positive Rate)')
plt.ylabel('真正率 (True Positive Rate)')
plt.title('ROC曲线 - RandomForest')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()

# 记录到ClearML
logger.report_matplotlib_figure(
    title="ROC曲线",
    series="roc_curve",
    figure=plt.gcf(),
    report_interactive=False
)

# 保存图像
roc_image_path = 'roc_curve_rf.png'
plt.savefig(roc_image_path, dpi=100, bbox_inches='tight')
plt.close()

print(f"✅ ROC曲线已记录 (AUC = {roc_auc:.2f}%)")

# 上传ROC曲线图像作为Artifact
task_ml2.upload_artifact(
    name='roc_curve_rf.png',
    artifact_object=roc_image_path
)
# ==================== 12. 保存模型为pkl文件 ====================
print("\n" + "="*50)
print("步骤8: 保存模型为pkl文件")

# 保存模型
model_filename = 'random_forest_model_v1.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ 模型已保存为: {model_filename}")

# 保存特征重要性
feature_importance_dict = {
    'feature_importances': feature_importance.tolist(),
    'top_features': []
}

# 识别最重要的特征
indices = np.argsort(feature_importance)[::-1]
for i, idx in enumerate(indices[:10]):  # 前10个最重要特征
    feature_importance_dict['top_features'].append({
        'feature_index': int(idx),
        'importance': float(feature_importance[idx]),
        'rank': i + 1
    })

importance_filename = 'feature_importance_rf.json'
with open(importance_filename, 'w') as f:
    json.dump(feature_importance_dict, f, indent=2)

print(f"✅ 特征重要性已保存为: {importance_filename}")

# ==================== 12. 上传模型和文件作为Artifact ====================
print("\n" + "="*50)
print("步骤9: 上传模型和文件到ClearML")

# 上传模型
task_ml2.upload_artifact(
    name='random_forest_model.pkl',
    artifact_object=model_filename,
    metadata={
        'model_type': 'RandomForestClassifier',
        'accuracy': accuracy,
        'hyperparameters': hyperparams
    }
)

# 上传特征重要性
task_ml2.upload_artifact(
    name='feature_importance_rf.json',
    artifact_object=importance_filename
)

# 上传混淆矩阵图像
task_ml2.upload_artifact(
    name='confusion_matrix_rf.png',
    artifact_object=cm_image_path
)

print("✅ 模型和文件已上传为Artifact")

# ==================== 13. 记录实验总结 ====================
print("\n" + "="*50)
print("步骤10: 记录实验总结")

# 创建实验总结
experiment_summary = {
    'experiment_info': {
        'task_id': task_ml2.id,
        'model': 'RandomForestClassifier',
        'dataset_version': '2.0',
        'timestamp': pd.Timestamp.now().isoformat()
    },
    'performance_metrics': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    },
    'model_info': {
        'n_estimators': model.n_estimators,
        'n_features': X_train.shape[1],
        'n_classes': len(np.unique(y_train)),
        'feature_importance_top_3': [
            (int(indices[0]), float(feature_importance[indices[0]])),
            (int(indices[1]), float(feature_importance[indices[1]])),
            (int(indices[2]), float(feature_importance[indices[2]]))
        ]
    },
    'hyperparameters_used': hyperparams
}

# 记录总结表格
summary_table = [
    ["指标", "值"],
    ["准确率", f"{accuracy:.2f}%"],
    ["精确率", f"{precision:.2f}%"],
    ["召回率", f"{recall:.2f}%"],
    ["F1分数", f"{f1:.2f}%"],
    ["树的数量", model.n_estimators],
    ["特征数量", X_train.shape[1]],
    ["最大深度", hyperparams['max_depth']],
    ["OOB分数", f"{model.oob_score_:.4f}" if hasattr(model, 'oob_score_') else "N/A"],
    ["数据集版本", "2.0"]
]

logger.report_table(
    title="实验总结",
    series="summary",
    table_plot=summary_table
)

# ==================== 14. 比较两个实验的指标 ====================
print("\n" + "="*50)
print("步骤11: 实验对比总结")

print(f"\n📊 AUC对比 (实验1 vs 实验2):")
exp1_auc = 86.73  
print(f"   实验1 (DecisionTree) AUC: {exp1_auc:.2f}%")
print(f"   实验2 (RandomForest) AUC: {roc_auc:.2f}%")
print(f"   改进: {roc_auc - exp1_auc:.2f}%")

exp1_accuracy = 82.58  # 实验1的准确率

comparison_table = [
    ["实验", "模型", "准确率","AUC", "精确率", "召回率", "F1分数", "特征数", "备注"],
    ["实验1", "DecisionTree", f"{exp1_accuracy:.2f}%", f"{exp1_auc:.2f}%","~82%", "~82%", "~82%", "12", "基准模型"],
    ["实验2", "RandomForest", f"{accuracy:.2f}%", f"{roc_auc:.2f}%",f"{precision:.2f}%", f"{recall:.2f}%", f"{f1:.2f}%", "12", "集成学习"]
]

logger.report_table(
    title="实验对比 (包含AUC)",
    series="comparison_with_auc",
    table_plot=comparison_table
)


print("\n📊 实验对比:")
print(f"   实验1 (DecisionTree): {exp1_accuracy:.2f}% 准确率")
print(f"   实验2 (RandomForest): {accuracy:.2f}% 准确率")
print(f"   差异: {accuracy - exp1_accuracy:.2f}%")

# ==================== 15. 清理临时文件 ====================
print("\n" + "="*50)
print("步骤12: 清理临时文件")

import os
import time

temp_files = [model_filename, importance_filename, cm_image_path, roc_image_path]
for file in temp_files:
    if os.path.exists(file):
        try:
            os.remove(file)
            print(f"🗑️  已删除临时文件: {file}")
        except PermissionError:
            print(f"⚠️  警告: 无法删除 {file}，文件可能被其他程序锁定")
            print(f"    请稍后手动删除此文件")
        except Exception as e:
            print(f"⚠️  警告: 删除 {file} 时出错: {e}")

# ==================== 16. 完成任务 ====================
print("\n" + "="*50)
print("🎉 第二个ML实验完成!")

print(f"\n📊 实验总结:")
print(f"   任务ID: {task_ml2.id}")
print(f"   任务链接: {task_ml2.get_output_log_web_page()}")
print(f"   模型: RandomForestClassifier")
print(f"   准确率: {accuracy:.2f}%")
print(f"   树的数量: {model.n_estimators}")
print(f"   数据集版本: 2.0")

print(f"\n📋 在ClearML中可以查看:")
print(f"   - Configuration: RandomForest超参数")
print(f"   - Artifacts: 模型文件 (.pkl)")
print(f"   - RESULTS → SCALARS: 准确率等指标")
print(f"   - RESULTS → PLOTS: 混淆矩阵")

# 关闭任务
task_ml2.close()
print("\n✅ 任务已关闭")
print("📋 请访问ClearML Web界面查看完整结果")

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=N7LJUE5KY47X5YIOQ0P62YWNM6LMMZ
env: CLEARML_API_SECRET_KEY=if0s4bvxfz8Vrlx9vXgd0_INSvzWlQCFcZfJb_N96WcUjbHb3xIOMYjHCWh2zipx6-g
初始化第二个ML实验任务...
✅ ML实验任务2创建成功!
📋 Task ID: 24ac78ed937240de89499b9e96513fa7

步骤1: 加载Dataset v2.0 (与实验1相同)
✅ Dataset v2.0 加载成功!
📂 本地路径: C:/Users/User/.clearml/cache/storage_manager/datasets/ds_a49570fdf5eb453498b94ce70baddd3f

步骤2: 加载训练和测试数据
✅ 数据加载完成:
   训练集: 34189 样本, 12 特征
   测试集: 14653 样本

步骤3: 设置并记录RandomForest超参数
✅ 超参数已记录到ClearML

📋 使用的主要超参数:
   model_type: RandomForestClassifier
   n_estimators: 100
   criterion: gini
   max_depth: 10
   min_samples_split: 2
   min_samples_leaf: 1
   max_features: sqrt
   bootstrap: True
   oob_score: True
   random_state: 42
   n_jobs: -1

步骤4: 训练RandomForest模型
开始训练RandomForest模型...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished


✅ 模型训练完成!
📊 OOB分数: 0.8614

🔍 特征重要性 (前10个):
   特征 5: 0.2457
   特征 8: 0.2370
   特征 3: 0.1167
   特征 0: 0.0978
   特征 2: 0.0809
   特征 9: 0.0684
   特征 10: 0.0570
   特征 4: 0.0537
   特征 7: 0.0215
   特征 1: 0.0115

步骤5: 模型评估


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


📊 模型评估结果:
   准确率 (Accuracy): 86.07%
   精确率 (Precision): 85.64%
   召回率 (Recall): 86.07%
   F1分数: 85.05%
   AUC分数: 91.23%

📋 分类报告:
              precision    recall  f1-score   support

       <=50K       0.87      0.96      0.91     11116
        >50K       0.81      0.55      0.65      3537

    accuracy                           0.86     14653
   macro avg       0.84      0.75      0.78     14653
weighted avg       0.86      0.86      0.85     14653


步骤6: 记录指标到ClearML
✅ 指标已记录到ClearML (RESULTS → SCALARS)

记录分类报告表格...

记录混淆矩阵...


C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 39044 (\N{CJK UNIFIED IDEOGRAPH-9884}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 27979 (\N{CJK UNIFIED IDEOGRAPH-6D4B}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 26631 (\N{CJK UNIFIED IDEOGRAPH-6807}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 31614 (\N{CJK UNIFIED IDEOGRAPH-7B7E}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 30495 (\N{CJK UNIFIED IDEOGRAPH-771F}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:255: UserWarning:

Glyph 23454 (\N{CJK UNIFIED IDEOGRAPH-5B9E}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\i

✅ 混淆矩阵已记录

步骤7: 记录ROC曲线


C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 30495 (\N{CJK UNIFIED IDEOGRAPH-771F}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 26354 (\N{CJK UNIFIED IDEOGRAPH-66F2}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 32447 (\N{CJK UNIFIED IDEOGRAPH-7EBF}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 38543 (\N{CJK UNIFIED IDEOGRAPH-968F}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 26426 (\N{CJK UNIFIED IDEOGRAPH-673A}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\ipykernel_20620\910761404.py:289: UserWarning:

Glyph 29468 (\N{CJK UNIFIED IDEOGRAPH-731C}) missing from font(s) DejaVu Sans.

C:\Users\User\AppData\Local\Temp\i

✅ ROC曲线已记录 (AUC = 91.23%)

步骤8: 保存模型为pkl文件
✅ 模型已保存为: random_forest_model_v1.pkl
✅ 特征重要性已保存为: feature_importance_rf.json

步骤9: 上传模型和文件到ClearML
✅ 模型和文件已上传为Artifact

步骤10: 记录实验总结

步骤11: 实验对比总结

📊 AUC对比 (实验1 vs 实验2):
   实验1 (DecisionTree) AUC: 86.73%
   实验2 (RandomForest) AUC: 91.23%
   改进: 4.50%

📊 实验对比:
   实验1 (DecisionTree): 82.58% 准确率
   实验2 (RandomForest): 86.07% 准确率
   差异: 3.49%

步骤12: 清理临时文件
⚠️  警告: 无法删除 random_forest_model_v1.pkl，文件可能被其他程序锁定
    请稍后手动删除此文件
🗑️  已删除临时文件: feature_importance_rf.json
🗑️  已删除临时文件: confusion_matrix_rf.png
🗑️  已删除临时文件: roc_curve_rf.png

🎉 第二个ML实验完成!

📊 实验总结:
   任务ID: 24ac78ed937240de89499b9e96513fa7
   任务链接: https://app.clear.ml/projects/f515e51c6fa94d259f4175808f6ac5d6/experiments/24ac78ed937240de89499b9e96513fa7/output/log
   模型: RandomForestClassifier
   准确率: 86.07%
   树的数量: 100
   数据集版本: 2.0

📋 在ClearML中可以查看:
   - Configuration: RandomForest超参数
   - Artifacts: 模型文件 (.pkl)
   - RESULTS → SCALARS: 准确率等指标
   - RESULTS → PLOTS: 混淆矩阵

✅ 任务已关闭
📋 请访问ClearML W

Skipping upload, could not find object file 'c:/Users/User/Desktop/机器学习学期作业/feature_importance_rf.json'
Skipping upload, could not find object file 'c:/Users/User/Desktop/机器学习学期作业/confusion_matrix_rf.png'
